In [2]:
import json
import os
import sys
import time
from itertools import combinations
from subprocess import Popen

import requests
import numpy as np

## Freesound search and download routines

In [ ]:
with open('token.json') as f:
    # {"oauth2_token": "token", "api_secret": "secret"}
    apikey = json.load(f)

headers = {
    'Authorization': 'Bearer ' + apikey['oauth2_token']
}

freesound_download_dir = 'data/freesound_whooshes'
text_search_url = 'http://www.freesound.org/apiv2/search/text/'
sound_download_url_tpl = 'https://www.freesound.org/apiv2/sounds/{}/download/'


def search_tag(tag):
    params = {
        'filter': tag,
        'token': apikey['api_secret']
    }
    sounds = []
    try:
        r = requests.get(text_search_url, params)
        sounds.extend([s['id'] for s in r.json()['results']])
        while r.json()['next'] is not None:
            r = requests.get(r.json()['next'], {'token': params['token']})
            sounds.extend([s['id'] for s in r.json()['results']])
            time.sleep(0.5)
    finally:
        with open(freesound_download_dir + '/sounds.json', 'w') as f:
            json.dump(sounds, f)


def download_sound(sid):
    r = requests.get(sound_download_url_tpl.format(sid), headers=headers)
    r.raw.decode_content = True
    filetype = r.headers['Content-Disposition'].split('.')[-1].replace('"', '')
    with open('{}/{}.{}'.format(freesound_download_dir, sid, filetype), 'wb') as f:
        f.write(r.content)

### Download sounds from a list

In [ ]:
with open('data/freesound_whoosh_list.txt', 'r') as f:
        sids = [int(x) for x in f.readlines()]
        for i, s in enumerate(sids):
            print('{}\t{}'.format(i, s))
            download_sound(s)
            time.sleep(1)

## Generate sine tones

In [ ]:
! csound gen_test_tones.csd --omacro:TEST_TYPE=1 -f --output=data/test_sines/sweep_down.wav

In [ ]:
! csound gen_test_tones.csd --omacro:TEST_TYPE=2 -f --output=data/test_sines/sweep_up.wav

In [ ]:
! csound gen_test_tones.csd --omacro:TEST_TYPE=3 -f --output=data/test_sines/sweep_both.wav

## Generate noises

In [4]:
def between(x1, x2):
    return int((x2 - x1) * 0.5) + x1


def filter_freq_pairs(tupl):
    (f1l, f1h), (f2l, f2h) = tupl
    c1 = between(f1l, f1h)
    c2 = between(f2l, f2h)
    return abs(c1 - c2) < 5000


def gen_freq_list():
    return [(int(f1), int(f2))
            for (f1, f2)
            in combinations(np.linspace(0, 20000, 100), 2)
            if f2 > f1]


def cmp_as_string(x, y):
    if x > y:
        return 'down'
    elif x < y:
        return 'up'
    else:
        return 'still'

In [5]:
freqs = gen_freq_list()
pair_freqs = filter(filter_freq_pairs,
                    combinations(freqs, 2))[::200]

In [50]:
total_combs = len(pair_freqs)

In [113]:
noises_dir = 'data/test_noises/'
cmd_template = 'csound gen_noises.csd --omacro:cf1={} --omacro:bw1={} ' \
               '--omacro:cf2={} --omacro:bw2={} -f --output={}'

def render_file(low1, high1, low2, high2):
    bw1 = high1 - low1
    cf1 = low1 + int(bw1 * 0.5)
    bw2 = high2 - low2
    cf2 = low2 + int(bw2 * 0.5)
    wav_name = '{}-{}--{}-{}--{}.wav'.format(cf1, bw1, cf2, bw2,
                                             cmp_as_string(cf1, cf2))
    if os.path.isfile(noises_dir + wav_name):
        return Popen('false')
    cmd = cmd_template.format(cf1, bw1, cf2, bw2, noises_dir + wav_name)
    return Popen(cmd.split(' '))


def run_parallel(pair_freq_list, num_threads=4):
    procs = set()
    count = 0
    for (low1, high1), (low2, high2) in pair_freq_list:
        count += 1
        sys.stdout.write('{} / {}\r'.format(count, total_combs))
        procs.add(render_file(low1, high1, low2, high2))
        if len(procs) > num_threads:
            os.wait()  # won't work on Windows
            procs -= {p for p in procs if p.poll() is not None}

In [114]:
run_parallel(pair_freqs)